In [1]:
%pip install --upgrade pymupdf PyPDF2

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 12.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from PyPDF2 import PdfReader
import re,fitz
import pandas as pd
import os

In [3]:
ocr = False
if ocr:
    from tempfile import TemporaryDirectory
    import pytesseract
    from pdf2image import convert_from_path
    from PIL import Image

In [4]:
#CHANGE FILENAME TO THE NEW ONE
FILENAME = 'Provas/OAB42.pdf'
GABARITO = 'Gabaritos/OAB42-GABARITO.pdf'
OUTNAME = 'OAB_42.csv'

In [64]:
class OABAutomata:
    def __init__(self):
        # Inicializa o estado da máquina como 0 (estado inicial).
        self.state = 0
        
        # Inicializa um dicionário que armazenará as perguntas.
        self.question = {}
        
        # Dicionário que mapeia os estados para descrições legíveis.
        self.state_dict = {
            0:'header', #número da questão
            1:'body', #questão
            'A':'alternative A',
            'B':'alternative B',
            'C':'alternative C',
            'D':'alternative D',
        }
        
    def clear_memory(self):
        # Limpa a memória da máquina, resetando o estado e as perguntas.
        self.state = 0
        self.question = {}
        
    def letter_state(self,current_state,next_state,part):
        # Verifica se a parte não corresponde ao próximo estado ou ao formato esperado.
        if part.strip() != next_state and part.strip() != next_state.lower()+'.':
            # Se o estado atual não estiver presente na pergunta, cria uma chave para ele.
            if current_state not in self.question:
                self.question[current_state] = ''
            # Adiciona a parte da pergunta ao estado atual, com formatação.
            self.question[current_state] += (part.strip('\n') + ' ').replace('  ', ' ')
        else:
            # Caso contrário, atualiza o estado para o próximo estado.
            self.state = next_state
        
    def read(self, part):
        # Estado final: checa se o estado atual tem um padrão que indica término da questão
        if self.state == 'D' and (
            re.search(r'^\d+$', part.strip()) or #Número da questão (string de número sozinho)
            re.search('Realização', part) or #Final da prova
            re.search('Tipo 1 -', part) or #Rodapé da página
            re.search('Página -', part) or #Rodapé da página
            re.search('FGV', part) #Rodapé da página
        ):
            # Cria uma cópia do dicionário de perguntas e limpa a memória.
            ret = self.question.copy()
            self.clear_memory()
            return ret
        
        # Se a parte estiver vazia, retorna False.
        if not part:
            return False
        
        # Se encontrar um número isolado no início e o estado for 0
        # armazena a pergunta no dicionário e passa para o estado 1.
        elif re.search('^\d+$', part.strip()) and self.state == 0:
            self.question['question'] = part.strip()
            self.state = 1
        
        # Se o estado for 1 e a parte não for uma alternativa (a primeira 'A')
        # adiciona a parte ao corpo da pergunta
        elif self.state == 1 and (part.strip() != 'A)' and part.strip() != 'a)'):
            if 'body' not in self.question:
                self.question['body'] = ''
            self.question['body'] += (part.strip('\n') + ' ').replace('  ', ' ') #replace pra remover espaços duplos
        
        # Se o estado for 1 e a parte for 'A'
        # passa para o próximo estado 'A'.
        elif self.state == 1:
            self.state = 'A' #estado A do dicionário, não string (ou seja, não precisa mudar pra 'A)')
        
        # Para outros estados (letras 'A' até 'D')
        # chama letter_state para adicionar conteúdo.
        elif self.state != 0 and self.state != 1:
            #função chr(ord(state+1) retorna o próximo caractere (de a para b)
            self.letter_state(self.state, chr(ord(self.state) + 1), part)
        
        # Retorna False por padrão, caso não haja correspondência com os estados.
        return False


In [65]:
class OCRAutomata:
    def __init__(self):
        # Inicializa a classe com dois atributos principais:
        self.state = 0  # Estado inicial (representa a fase atual do processamento).
        self.question = {}  # Dicionário para armazenar as informações da questão atual.
        
        # Dicionário para mapeamento de estados a seus significados (usado para facilitar o entendimento do código).
        self.state_dict = {
            0: 'header',          # Estado 0: cabeçalho ou antes de uma questão.
            1: 'body',            # Estado 1: corpo da questão.
            'A': 'alternative A', # Alternativa A.
            'B': 'alternative B', # Alternativa B.
            'C': 'alternative C', # Alternativa C.
            'D': 'alternative D', # Alternativa D.
        }

    def clear_memory(self):
        # Limpa os dados armazenados para preparar o automato para uma nova questão.
        self.state = 0  # Reseta o estado para o inicial (0).
        self.question = {}  # Reseta o dicionário da questão.
        
        # Inicializa os campos padrão para a questão.
        self.question['question'] = ''  # Título ou número da questão.
        self.question['body'] = ''      # Corpo ou enunciado da questão.
        self.question['A'] = None       # Alternativa A.
        self.question['B'] = None       # Alternativa B.
        self.question['C'] = None       # Alternativa C.
        self.question['D'] = None       # Alternativa D.

    def read(self, part):
        # Função principal que processa uma parte (linha ou bloco) do texto extraído.
        
        # Checa se o estado é 1 e se encontra padrões que indicam o início de uma nova questão
        # ou o fim da questão atual (palavras-chave como "ENDOFENEM", "LC -", etc.).
        if self.state == 1 and (re.search(r'^\d+$', part.strip()) or #Número da questão (string de número sozinho)
                                re.search('Realização', part) or #Final da prova
                                re.search('Tipo 1 -', part) or #Rodapé da página
                                re.search('Página -', part) or #Rodapé da página
                                re.search('FGV', part)): #Rodapé da página
            ret = self.question.copy()  # Copia os dados da questão atual.
            self.clear_memory()  # Reseta para processar a próxima questão.
            return ret  # Retorna a questão processada.
        
        # Se a parte está vazia, não faz nada e retorna `False`.
        if not part:
            return False

        # Checa se o estado é 0 (inicial) e encontra o padrão "questão [número]".
        elif re.search('^\d+$', part.lower()) and self.state == 0:
            # Armazena o número da questão no dicionário e muda o estado para 1 (corpo da questão).
            self.question['question'] = re.search('^\d+$', part.lower()).group()
            self.state = 1

        # Se o estado for 1 (corpo da questão), adiciona a parte ao campo "body".
        elif self.state == 1:
            if 'body' not in self.question:
                self.question['body'] = ''  # Garante que o campo "body" existe.
            self.question['body'] += (part.strip('\n') + ' ').replace('  ', ' ')  # Remove quebras de linha e espaços extras.

        return False  # Retorna `False` quando nenhuma outra condição é atendida.

In [70]:
class PhysicalEnemParser:
    def __init__(self, enem_object, engine='pypdf2'):
        # Inicializa a classe com dois parâmetros principais:
        # enem_object: o objeto que contém o arquivo PDF (pode ser de diversos tipos).
        # engine: o motor de extração de texto a ser utilizado (por padrão, 'pypdf2').
        self.enem_object = enem_object
        self.engine = engine
        parts = []  # Lista onde o texto extraído será armazenado.

        # Caso o motor seja 'pymupdf', usa a biblioteca PyMuPDF para extrair o texto.
        if engine == 'pymupdf':
            for page_num in range(1, len(enem_object)):
                page = enem_object[page_num]  # Página atual do PDF.
                image_list = page.get_images(full=True)  # Obtém todas as imagens da página.
                to_remove = []  # Lista para armazenar textos que serão removidos (provenientes de imagens).
                
                # Para cada imagem, obtém a caixa delimitadora e o texto da imagem.
                for image in image_list:
                    bbox = page.get_image_bbox(image)  # Obtém a caixa delimitadora da imagem.
                    tb = page.get_textbox(bbox)  # Obtém o texto na caixa delimitadora.
                    to_remove.extend(tb.split('\n'))  # Divide o texto em linhas e adiciona à lista de remoção.
                
                page_text = page.get_text().split('\n')  # Obtém o texto da página (dividido em linhas).
                
                # Adiciona à lista `parts` as linhas de texto que não estão na lista `to_remove`.
                for text in page_text:
                    if text not in to_remove:
                        parts.append(text)

        # Caso o motor seja 'pypdf2', usa a biblioteca PyPDF2 para extrair o texto.
        if engine == 'pypdf2':
            def visitor_body(text, cm, tm, fontDict, fontSize):
                # Função auxiliar para processar o texto extraído.
                parts.append(text.replace('[supressão de texto]', '[...]'))  # Substitui um texto específico.

            # Itera pelas páginas do PDF extraído.
            for page in enem_object.pages:
                page.extract_text(visitor_text=visitor_body)  # Extraí o texto de cada página.
            parts.append('ENDOFENEM')  # Adiciona um marcador de fim de documento.

        # Caso o motor seja 'OCR', usa OCR (Reconhecimento Óptico de Caracteres) para extrair o texto.
        if engine == 'OCR':
            language_config = r'-l por --psm 1'  # Configuração de idioma e modo do OCR.
            PDF_file = enem_object  # Caminho do arquivo PDF.
            image_file_list = []  # Lista para armazenar os nomes das imagens geradas.

            with TemporaryDirectory() as tempdir:
                # Cria um diretório temporário para armazenar as imagens temporárias.
                pdf_pages = convert_from_path(PDF_file, 500)  # Converte as páginas do PDF em imagens.

                # Para cada página, salva a imagem em um arquivo temporário.
                for page_enumeration, page in enumerate(pdf_pages, start=1):
                    filename = f"{tempdir}\\page_{page_enumeration:03}.jpg"  # Nome do arquivo de imagem.
                    page.save(filename, "JPEG")  # Salva a imagem como JPEG.
                    image_file_list.append(filename)  # Adiciona à lista de arquivos de imagem.

                parsed = ''  # Variável para armazenar o texto reconhecido.

                # Para cada arquivo de imagem, usa o OCR para extrair o texto.
                for image_file in image_file_list:
                    text = str(((pytesseract.image_to_string(Image.open(image_file), config=language_config))))
                    parsed += text  # Adiciona o texto extraído à variável `parsed`.

                parts = parsed.split('\n')  # Divide o texto extraído em linhas.

        self.parts = parts  # Atribui a lista de partes extraídas ao atributo da classe.

    def parse_questions(self):
        # Função que processa as partes extraídas e converte em questões.
        
        # Inicializa o automato que será usado para interpretar as questões.
        self.automata = OABAutomata()
        
        # Caso o motor seja OCR, usa um automato OCRAutomata para lidar com questões de redação.
        if self.engine == 'OCR':
            self.automata = OCRAutomata()

        questions = []  # Lista para armazenar as questões processadas.

        # Para cada parte extraída do texto, tenta interpretar e identificar questões.
        for part in self.parts:
            accept = self.automata.read(part)  # Lê e interpreta a parte.
            
            # Enquanto o automato reconhecer e aceitar partes válidas, adiciona-as à lista de questões.
            while accept:
                questions.append(accept)
                accept = self.automata.read(part)

        return questions  # Retorna as questões processadas.

In [71]:
# Lista todos os arquivos na pasta 
files = list(os.listdir('Provas'))

# Itera sobre cada arquivo na pasta 
for fileno in files:
    # Define os caminhos dos arquivos de entrada (PDF) e saída (CSV)
    FILENAME = 'Provas/' + fileno  # Caminho do arquivo PDF
    OUTNAME = 'Data/' + fileno.strip('.pdf') + '.csv'  # Nome do arquivo CSV de saída (removendo a extensão '.pdf')

    # Lê o arquivo PDF usando o PyPDF2 (PdfReader)
    oab = PdfReader(FILENAME)  # Função da biblioteca PyPDF2 que carrega o arquivo PDF
    # Cria um parser para processar o conteúdo do PDF
    parser = PhysicalEnemParser(oab, engine='pypdf2')
    
    # Extrai as questões do PDF processado
    questions = parser.parse_questions()

    # Cria um DataFrame a partir das questões extraídas
    df = pd.DataFrame(questions)
    
    # Para cada coluna no DataFrame
    for column in df.columns:
        # Aplica uma limpeza no texto de cada célula, substituindo tabulações por espaços, removendo espaços extras e fazendo strip
        df[column] = df[column].apply(lambda x: x.replace('\t', ' ').replace('  ', ' ').strip())
    
    # Salva o DataFrame processado em um arquivo CSV no diretório 'Data'
    df.to_csv(OUTNAME, index=False)  # Salva o DataFrame como CSV sem incluir o índice